Este programa pega o conteúdo do livro de três formas diferentes, que se pode escolher: 
- **Método 1:** Um .html com todo o material (separação de capítulos por títulos dentro do documento)
    - Use a função do GoogleDocs para exportar HTML (Arquivo > Fazer Download > Página Web), ele deixa mais enxuto o documento e garante que o programa funcione. 
- **Método 2:** Importação de Múltiplos .html (um por capítulo)
    - Ainda é uma boa ideia usar o GoogleDocs, mas se usar de outra ferramenta (conversores, Microsoft Word, etc...) ainda pode ser usado.
- **Método 3:** Web Scraping do material de sites de agregação de capítulos
    - Buscar um capítulo em cada página web

## Configuração de Bibliotecas e Variáveis do Programa.

In [312]:
# Não mexa aqui
from genericpath import isdir
from urllib.request import urlopen
from bs4 import BeautifulSoup
from ebooklib import epub
import base64
import requests
import shutil
import os
import json
import re

# Escolha do método para colocar o material no livro (vide "Passo 1")
metodo = 1

# ============================ Configurações do Método 1 ==========================

# Caminho do arquivo .html com todo material
caminho_html = "Bakegoogle.html"

# Quais tipos de títulos serão usados para separar cada capítulo (h2 = título 2)
separadores = [
    # "h1",
    "h2",
    # "h3",
    # "h4",
    # "h5",
    # "h6",
]

# ============================ Configurações do Método 2 ==========================

# Caminho da pasta com os arquivos .html (lidos em ordem alfabética)
caminho_pasta = ""


# ============================ Configurações do Método 3 ==========================

# Lista de URLs para pegar os capitulos (separar cada URL com vírgula dentro das chaves)
urls = [
    "https://github.com/",
    "https://youtube.com/",
    "https://google.com/",
]


# ============================= Configurações do Ebook ===========================

# Uma linha de identificação única, pode ser o que você quiser
identificador = "12349495893"

titulo = "Pakepopogatari"
autor = "Pipioipin"
idioma = "pt-br"

# Caminho do arquivo de imagem da capa
capa = "reacao.jpg"

# Uma descrição do livro
descricao = "Janeeeeeeee"

## Funções Auxiliares

In [313]:
def limpa_texto(text):
    # Texto Bonitinho :)
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip()
                for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

# def dataurl(filename):
#     item = {}
#     ext = filename.split('.')[-1]
#     prefix = f'data:image/{ext};base64,'
#     with open(filename, 'rb') as f:
#         img = f.read()

#     return prefix + base64.b64encode(img).decode('utf-8')


## Passo 1: Pegar o material


#### Método 1:


In [314]:
soup = None
if metodo == 1:
    with open(caminho_html, "r", encoding="utf-8") as reader:
        html = reader.read()

    soup = BeautifulSoup(html, features="html.parser")

elif metodo == 2:
    print()

elif metodo == 3:
    print()


### Passo 2: Tratar o material 




In [315]:
# Pega o conteúdo do body do html para iterar
body = soup.find("body")
children = body.findChildren(recursive=False)

# Cria uma tupla de tags de título que podem começar um novo capítulo ("<h2", "<h3" ...)
tags = tuple(f"<{tag}" for tag in separadores)

capitulos = []
titulos = []
cap = ""
title = "Intro"
for child in children:
    content = str(child)
    # Começa um novo capítulo conforme as tags de novo cap
    if content.startswith(tags):
        # Se o header não tiver nada escrito, continua o loop
        if not child.text.strip():
            continue 

        # Confirmação de que não tem um capítulo vazio poluindo a lista
        if cap:
            capitulos.append(cap)
            titulos.append(title.strip())
            cap = ""
            title = child.text
        cap += content
    else: 
        # Adiciona o conteúdo na página html
        cap += content

capitulos = [limpa_texto(cap) for cap in capitulos]
titulos = [limpa_texto(tit) for tit in titulos]


estilo = ""
for blob in soup.findAll("style"):
    estilo += blob.contents[0]


### Passo 3: Criação do Ebook

In [316]:
def estiloPadrao():
    return '''
            @namespace epub "http://www.idpf.org/2007/ops";
            body {
                font-family: Cambria, Liberation Serif, Bitstream Vera Serif, Georgia, Times, Times New Roman, serif;
            }
            h2 {
                text-align: left;
                text-transform: uppercase;
                font-weight: 200;     
            }
            ol {
                    list-style-type: none;
            }
            ol > li:first-child {
                    margin-top: 0.3em;
            }
            nav[epub|type~='toc'] > ol > li > ol  {
                list-style-type:square;
            }
            nav[epub|type~='toc'] > ol > li > ol > li {
                    margin-top: 0.3em;
            }
            body { 
                font-family: Times, 
                Times New Roman, serif; 
            }
        '''

In [317]:
book = epub.EpubBook()

# Metadata do Ebook
book.set_identifier(identificador)
book.set_title(titulo)
book.add_author(autor)
book.set_language(idioma)
book.add_metadata('DC', 'description', descricao)
book.set_cover("cover.png", open(capa, 'rb').read())

# Faz estilos
if not estilo:
    estilo = estiloPadrao()

# Adiciona estilos
css = epub.EpubItem(uid="style_nav",
                        file_name="style.css",
                        media_type="text/css",
                        content=estilo)
book.add_item(css)

cover = epub.EpubHtml(uid="front-cover",
                    title="Capa",
                    file_name=f'front-cover.xhtml',
                    lang=idioma)
cover.set_content(f"<img src=\"cover.png\" alt=\"Cover\"/>")
book.add_item(cover)


spine = [cover, "nav"]
toc = []
for cap, title in zip(capitulos, titulos):
    # Create a chapter and fill it with HTML and css
    c = epub.EpubHtml(uid=title,
                        title=title,
                        file_name=f'{title}.xhtml',
                        lang=idioma)
                        
    c.set_content(cap)
    c.add_item(css)
    book.add_item(c)

    # Criando a ordem de capítulos 
    spine.append(c)
    # Criando a lista de conteúdos 
    toc.append(epub.Link(f'{title}.xhtml', title, title)) 


spine.append("cover")

# data["images"] = list(set(data['images']))
# for img in data['images']:
#     p = epub.EpubItem(file_name=f"images/{img}", content=open('__img_data__\\' +
#                         img, 'rb').read(), media_type='image/' + img.split(".")[-1])
#     book.add_item(p)

# p1 = epub.EpubItem(file_name='images/artefatoDivino.png', content=open('__img_data__\\artefatoDivino.png', 'rb').read(), media_type='image/png')
# p2 = epub.EpubItem(file_name='images/Imagem_MT_07.png', content=open('__img_data__\\Imagem_MT_07.png', 'rb').read(), media_type='image/png')
# book.add_item(p1)
# book.add_item(p2)


# Ordem dos capítulos
book.spine = spine

# How to create the table of contents (it is a tuple/list)
spine = spine[2:len(spine)]

# display(spine)

# toc = []


book.toc = tuple(toc)

display(book.toc)

# Adiciona essas merdas aí, quem liga lol
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# Salva o livro
epub.write_epub(f"{titulo}.epub", book)





(<ebooklib.epub.Link at 0x1ea72aa96c0>,
 <ebooklib.epub.Link at 0x1ea72aa95d0>)